# Modern Portfolio Theory

- Modern Portfolio Theory (MPT) is a Nobel Prize-winning economic theory. 
- It explains how risk-averse investors can construct portfolios to optimize or maximize expected return based on a given level of market risk. 
- <a href = "https://en.wikipedia.org/wiki/Harry_Markowitz">Harry Markowitz</a> pioneered this theory in his paper <a href = "https://onlinelibrary.wiley.com/doi/full/10.1111/j.1540-6261.1952.tb01525.x">Portfolio Selection</a> , which was published in the Journal of Finance in 1952. He was later awarded a Nobel Prize for his work on modern portfolio theory.
- Modern Portfolio Theory suggests diversification of all your securities and asset classes and not putting all your eggs in one basket. It emphasises the importance of portfolios, diversification, risk and the connections among different kinds of securities.



In [1]:
pip install yfinance pandas numpy matplotlib requests beautifulsoup4 --break-system-packages

  Obtaining dependency information for yfinance from https://files.pythonhosted.org/packages/73/b5/d50eec88bc731bb8570ae42a9b764a36144e217361c33fa068391ff59ba3/yfinance-0.2.61-py2.py3-none-any.whl.metadata
  Obtaining dependency information for multitasking>=0.0.7 from https://files.pythonhosted.org/packages/3e/8a/bb3160e76e844db9e69a413f055818969c8acade64e1a9ac5ce9dfdcf6c1/multitasking-0.0.11-py3-none-any.whl.metadata
  Obtaining dependency information for frozendict>=2.3.4 from https://files.pythonhosted.org/packages/04/13/d9839089b900fa7b479cce495d62110cddc4bd5630a04d8469916c0e79c5/frozendict-2.4.6-py311-none-any.whl.metadata
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
      --------------------------------------- 0.1/3.0 MB 1.7 MB/s eta 0:00:02
     --- ------------------------------------ 0.3/3.0 MB 3.4 MB/s eta 0:00:01
     ------------- -------------------------- 1.0/3.0 MB 8.1 MB/s eta 0:00:01
     ------------------------- -------------- 2.0/3.0 MB 1

In [2]:
import yfinance as yf
import requests
import pandas as pd 

nifty50 = yf.download("^NSEI", start="2019-01-01", end="2025-03-31")['Close']

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [3]:
nifty50 = nifty50.rename(columns = {'^NSEI': 'Nifty100'})
nifty50.head()

Ticker,Nifty100
Date,
2019-01-02,10792.500000
2019-01-03,10672.250000
2019-01-04,10727.349609
2019-01-07,10771.799805
2019-01-08,10802.150391


In [4]:
def fetch_mutual_fund_nav(api_url, name):
    """
    Fetches mutual fund NAV data from mfapi.in and returns a single-column
    DataFrame with 'date' as index and the NAV under the provided name.
    """
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        data = response.json().get("data", [])

        # Load into DataFrame
        df = pd.DataFrame(data)[["date", "nav"]]

        # Convert types
        df["date"] = pd.to_datetime(df["date"], format="%d-%m-%Y")
        df["nav"] = df["nav"].astype(float)

        # Filter date range
        mask = (df["date"] >= "2019-01-01") & (df["date"] <= "2025-04-09")  # today
        df = df.loc[mask]

        # Final cleanup: set index and rename column
        df.set_index("date", inplace=True)
        df.rename(columns={"nav": name}, inplace=True)

        return df

    except Exception as e:
        print(f"Error: {e}")
        return pd.DataFrame(columns=[name])


In [5]:
parag = fetch_mutual_fund_nav("https://api.mfapi.in/mf/122639", "parag")
hdfc = fetch_mutual_fund_nav("https://api.mfapi.in/mf/118955", "hdfc")
icici = fetch_mutual_fund_nav("https://api.mfapi.in/mf/120596", "icici")
motilal = fetch_mutual_fund_nav("https://api.mfapi.in/mf/147704", "motilal")
parag = fetch_mutual_fund_nav("https://api.mfapi.in/mf/122639", "")

In [6]:
dates = pd.date_range(start="2019-01-01", end="2025-04-10", freq="M")
monthly_fd_returns = [(1 + 0.06) ** (1/12) - 1] * len(dates)
fd_df = pd.DataFrame(data=monthly_fd_returns, index=dates, columns=["FD_Return"])
fd_df.head()

,FD_Return
2019-01-31,0.004868
2019-02-28,0.004868
2019-03-31,0.004868
2019-04-30,0.004868
2019-05-31,0.004868


In [7]:
import yfinance as yf

# Download Close price and rename the Series
btc = yf.download("BTC-USD", start="2019-01-01", end="2025-03-31")["Close"]
btc.name = "Bitcoin"

eth = yf.download("ETH-USD", start="2019-01-01", end="2025-03-31")["Close"]
eth.name = "Ethereum"

sol = yf.download("SOL-USD", start="2021-01-01", end="2025-03-31")["Close"]
sol.name = "Solana"

# Combine into one DataFrame
crypto_df = pd.concat([btc, eth, sol], axis=1)
print(crypto_df.head())


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Ticker          BTC-USD     ETH-USD  SOL-USD
Date                                        
2019-01-01  3843.520020  140.819412      NaN
2019-01-02  3943.409424  155.047684      NaN
2019-01-03  3836.741211  149.135010      NaN
2019-01-04  3857.717529  154.581940      NaN
2019-01-05  3845.194580  155.638596      NaN


In [8]:

# Download daily gold closing prices
gold_df = yf.download("GC=F", start="2019-01-01", end="2025-03-31")[["Close"]]
gold_df = gold_df.rename(columns={"Close": "Gold"})

# Reset index to have 'date' as a column
gold_df = gold_df.reset_index()

# Optional: format date
gold_df["Date"] = gold_df["Date"].dt.date

# Final DataFrame
print(gold_df.head())


[*********************100%***********************]  1 of 1 completed

Price         Date         Gold
Ticker                     GC=F
0       2019-01-02  1281.000000
1       2019-01-03  1291.800049
2       2019-01-04  1282.699951
3       2019-01-07  1286.800049
4       2019-01-08  1283.199951


In [9]:
gold_df = gold_df.reset_index(drop=True)
gold_df.head()

Price,Date,Gold
Ticker,,GC=F
0,2019-01-02,1281.000000
1,2019-01-03,1291.800049
2,2019-01-04,1282.699951
3,2019-01-07,1286.800049
4,2019-01-08,1283.199951


In [10]:
gold_df.columns = gold_df.columns.get_level_values(-1)  # Keep only bottom level column names
gold_df.head()

Ticker,,GC=F
0,2019-01-02,1281.000000
1,2019-01-03,1291.800049
2,2019-01-04,1282.699951
3,2019-01-07,1286.800049
4,2019-01-08,1283.199951


In [11]:
combined = pd.concat([
    nifty50,
    parag,
    hdfc,
    icici,
    motilal,
    btc,
    eth,
    sol, 
    gold_df
], axis=1)


In [14]:
combined.tail(20)

,Nifty100,,hdfc,icici,motilal,BTC-USD,ETH-USD,SOL-USD,,GC=F
1550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-03,2890.199951
1551,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-04,2909.600098
1552,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-05,2915.300049
1553,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-06,2916.600098
1554,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-07,2904.699951
1555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-10,2891.000000
1556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-11,2912.899902
1557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-12,2939.100098
1558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-13,2984.300049
1559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-14,2994.500000


In [13]:
print(parag.columns)

Index([''], dtype='object')
